# Parsing a page with BeautifulSoup

In [5]:
import requests
page = requests.get("http://dataquestio.github.io/web-scraping-pages/simple.html")
page

<Response [200]>

In [7]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(page.content, 'html.parser')
print(soup.prettify())

<!DOCTYPE html>
<html>
 <head>
  <title>
   A simple example page
  </title>
 </head>
 <body>
  <p>
   Here is some simple content for this page.
  </p>
 </body>
</html>


In [8]:
list(soup.children)

['html', '\n', <html>
 <head>
 <title>A simple example page</title>
 </head>
 <body>
 <p>Here is some simple content for this page.</p>
 </body>
 </html>]

In [9]:
[type(item) for item in list(soup.children)]

[bs4.element.Doctype, bs4.element.NavigableString, bs4.element.Tag]

In [10]:
html = list(soup.children)[2]
list(html.children)

['\n', <head>
 <title>A simple example page</title>
 </head>, '\n', <body>
 <p>Here is some simple content for this page.</p>
 </body>, '\n']

In [11]:
body = list(html.children)[3]
list(body.children)

['\n', <p>Here is some simple content for this page.</p>, '\n']

In [12]:
p = list(body.children)[1]
p.get_text()

'Here is some simple content for this page.'

# Finding all instances of a tag at once

In [13]:
soup = BeautifulSoup(page.content, 'html.parser')
soup.find_all('p')

[<p>Here is some simple content for this page.</p>]

In [14]:
soup.find_all('p')[0].get_text()

'Here is some simple content for this page.'

In [15]:
soup.find('p')

<p>Here is some simple content for this page.</p>

In [16]:
page = requests.get("http://dataquestio.github.io/web-scraping-pages/ids_and_classes.html")
soup = BeautifulSoup(page.content, 'html.parser')
soup

<html>
<head>
<title>A simple example page</title>
</head>
<body>
<div>
<p class="inner-text first-item" id="first">
                First paragraph.
            </p>
<p class="inner-text">
                Second paragraph.
            </p>
</div>
<p class="outer-text first-item" id="second">
<b>
                First outer paragraph.
            </b>
</p>
<p class="outer-text">
<b>
                Second outer paragraph.
            </b>
</p>
</body>
</html>

In [17]:
soup.find_all('p', class_='outer-text')

[<p class="outer-text first-item" id="second">
 <b>
                 First outer paragraph.
             </b>
 </p>, <p class="outer-text">
 <b>
                 Second outer paragraph.
             </b>
 </p>]

In [18]:
soup.find_all(class_="outer-text")

[<p class="outer-text first-item" id="second">
 <b>
                 First outer paragraph.
             </b>
 </p>, <p class="outer-text">
 <b>
                 Second outer paragraph.
             </b>
 </p>]

In [19]:
soup.find_all(id="first")

[<p class="inner-text first-item" id="first">
                 First paragraph.
             </p>]

In [20]:
soup.select("div p")

[<p class="inner-text first-item" id="first">
                 First paragraph.
             </p>, <p class="inner-text">
                 Second paragraph.
             </p>]

# Scraping weather data

In [21]:
page = requests.get("http://forecast.weather.gov/MapClick.php?lat=37.7772&lon=-122.4168")
soup = BeautifulSoup(page.content, 'html.parser')
seven_day = soup.find(id="seven-day-forecast")
forecast_items = seven_day.find_all(class_="tombstone-container")
tonight = forecast_items[0]
print(tonight.prettify())

<div class="tombstone-container">
 <p class="period-name">
  Overnight
  <br/>
  <br/>
 </p>
 <p>
  <img alt="Overnight: Partly cloudy, with a low around 55. West southwest wind 9 to 11 mph. " class="forecast-icon" src="newimages/medium/nsct.png" title="Overnight: Partly cloudy, with a low around 55. West southwest wind 9 to 11 mph. "/>
 </p>
 <p class="short-desc">
  Partly Cloudy
 </p>
 <p class="temp temp-low">
  Low: 55 °F
 </p>
</div>


In [22]:
period = tonight.find(class_="period-name").get_text()
short_desc = tonight.find(class_="short-desc").get_text()
temp = tonight.find(class_="temp").get_text()
print(period)
print(short_desc)
print(temp)

Overnight
Partly Cloudy
Low: 55 °F


In [23]:
img = tonight.find("img")
desc = img['title']
print(desc)

Overnight: Partly cloudy, with a low around 55. West southwest wind 9 to 11 mph. 


# Scraping all weather data

In [24]:
period_tags = seven_day.select(".tombstone-container .period-name")
periods = [pt.get_text() for pt in period_tags]
periods

['Overnight',
 'Wednesday',
 'WednesdayNight',
 'Thursday',
 'ThursdayNight',
 'Friday',
 'FridayNight',
 'IndependenceDay',
 'SaturdayNight']

In [28]:
short_descs = [sd.get_text() for sd in seven_day.select(".tombstone-container .short-desc")]
temps = [t.get_text() for t in seven_day.select(".tombstone-container .temp")]
descs = [d["title"] for d in seven_day.select(".tombstone-container img")]
print(short_descs)
print(temps)
print(descs)

['Partly Cloudy', 'Mostly Sunny', 'Partly Cloudy', 'Mostly Sunny', 'Partly Cloudy', 'Mostly Sunny', 'Mostly Clear', 'Sunny', 'Clear']
['Low: 55 °F', 'High: 68 °F', 'Low: 55 °F', 'High: 69 °F', 'Low: 55 °F', 'High: 68 °F', 'Low: 55 °F', 'High: 72 °F', 'Low: 56 °F']
['Overnight: Partly cloudy, with a low around 55. West southwest wind 9 to 11 mph. ', 'Wednesday: Mostly sunny, with a high near 68. Southwest wind 7 to 15 mph, with gusts as high as 20 mph. ', 'Wednesday Night: Partly cloudy, with a low around 55. Southwest wind 9 to 14 mph, with gusts as high as 18 mph. ', 'Thursday: Mostly sunny, with a high near 69. Southwest wind 7 to 16 mph, with gusts as high as 21 mph. ', 'Thursday Night: Partly cloudy, with a low around 55. West southwest wind 9 to 16 mph, with gusts as high as 21 mph. ', 'Friday: Mostly sunny, with a high near 68.', 'Friday Night: Mostly clear, with a low around 55.', 'Independence Day: Sunny, with a high near 72.', 'Saturday Night: Clear, with a low around 56.']


# Combining our data into a Pandas Dataframe

In [32]:
import pandas as pd
weather = pd.DataFrame({
    "period": periods,
    "short_desc": short_descs,
    "temp": temps,
    "desc":descs
})
weather

,period,short_desc,temp,desc
0,Overnight,Partly Cloudy,Low: 55 °F,"Overnight: Partly cloudy, with a low around 55..."
1,Wednesday,Mostly Sunny,High: 68 °F,"Wednesday: Mostly sunny, with a high near 68. ..."
2,WednesdayNight,Partly Cloudy,Low: 55 °F,"Wednesday Night: Partly cloudy, with a low aro..."
3,Thursday,Mostly Sunny,High: 69 °F,"Thursday: Mostly sunny, with a high near 69. S..."
4,ThursdayNight,Partly Cloudy,Low: 55 °F,"Thursday Night: Partly cloudy, with a low arou..."
5,Friday,Mostly Sunny,High: 68 °F,"Friday: Mostly sunny, with a high near 68."
6,FridayNight,Mostly Clear,Low: 55 °F,"Friday Night: Mostly clear, with a low around 55."
7,IndependenceDay,Sunny,High: 72 °F,"Independence Day: Sunny, with a high near 72."
8,SaturdayNight,Clear,Low: 56 °F,"Saturday Night: Clear, with a low around 56."


In [30]:
temp_nums = weather["temp"].str.extract("(?P<temp_num>\d+)", expand=False)
weather["temp_num"] = temp_nums.astype('int')
temp_nums

0    55
1    68
2    55
3    69
4    55
5    68
6    55
7    72
8    56
Name: temp_num, dtype: object

In [31]:
weather["temp_num"].mean()

61.44444444444444

In [33]:
is_night = weather["temp"].str.contains("Low")
weather["is_night"] = is_night
is_night

0     True
1    False
2     True
3    False
4     True
5    False
6     True
7    False
8     True
Name: temp, dtype: bool

In [34]:
weather[is_night]

,period,short_desc,temp,desc,is_night
0,Overnight,Partly Cloudy,Low: 55 °F,"Overnight: Partly cloudy, with a low around 55...",True
2,WednesdayNight,Partly Cloudy,Low: 55 °F,"Wednesday Night: Partly cloudy, with a low aro...",True
4,ThursdayNight,Partly Cloudy,Low: 55 °F,"Thursday Night: Partly cloudy, with a low arou...",True
6,FridayNight,Mostly Clear,Low: 55 °F,"Friday Night: Mostly clear, with a low around 55.",True
8,SaturdayNight,Clear,Low: 56 °F,"Saturday Night: Clear, with a low around 56.",True
